In [ ]:
import os

# from src.customModel.customSACTrainer import KLSAC
dataset_path = "/workspace/datasets/"
os.environ["L5KIT_DATA_FOLDER"] = dataset_path
# os.environ['CUDA_VISIBLE_DEVICES']= '0'
# os.environ["TUNE_RESULT_DIR"] =  '/DATA/l5kit/rllib_tb_logs'
import gym
from l5kit.configs import load_config_data
from l5kit.environment.envs.l5_env import SimulationConfigGym, GymStepOutput, L5Env
# from l5kit.environment.envs.l5_env2 import SimulationConfigGym, GymStepOutput, L5Env2
from l5kit.visualization.visualizer.zarr_utils import episode_out_to_visualizer_scene_gym_cle
from l5kit.visualization.visualizer.visualizer import visualize
from bokeh.io import output_notebook, show
from l5kit.environment.gym_metric_set import L2DisplacementYawMetricSet, CLEMetricSet
from prettytable import PrettyTable
import datetime
from ray.rllib.models import ModelCatalog
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
import torch.nn as nn
import numpy as np
import gym
from typing import Dict
import numpy as np
import ray
import pytz
from ray import tune


from l5kit.configs import load_config_data

# get environment config
# env_config_path = '/workspace/source/configs/gym_config_history3.yaml'
# env_config_path = '/workspace/source/configs/gym_config84.yaml'
# env_config_path = SRC_PATH + 'src/configs/gym_vectorizer_config_hist3.yaml'

SRC_PATH = '/home/pronton/rlhf-car/'

import pickle
name = 'SAC-T-noFT-1e6_cp520(-57)'
name = 'KLSAC-T_nonRLFT_fixedKLKin_fixedConfig_KLweight=1_cp130(-51)'
name = 'BC-T-kin'

SAVE_PATH = f'{SRC_PATH}src/validate/testset/'
# file = open(f'{SAVE_PATH}{name}.obj', 'rb')
# sim_outs2 = pickle.load(file)
file = open(f'{SAVE_PATH}{name}_actions.obj', 'rb')
actions, indices = pickle.load(file)

cp = '150'
r = -82
name = 'KLRewardSAC-T_load_KLweight=.5_trainset_2e5_steer=-1.5'
name = f'KLRewardSAC-T_load_KLweight=.5_trainset_1e5_steer=-1.5'
name = f'KLRewardSAC-T_load_KLweight=.8_trainset_1e5_steer=-1.5'
name = f'KLRewardSAC-T_load_notfixedL5env2_KLweight=.3_trainset_1e5_acc_steer=-1.5'
cp = '160'
r = -90
name = f'KLSAC-T_nonRLFT_fixedKLKin_fixedConfig_KLweight=1'
name = f'KLSAC-T_load_acc=-1.5_MTAU=entropy_alpha_M_ALPHA=0.3_batch128'
name = f'SAC-T_load_trainset_2e5-1hist'
name = 'KLSAC-T_load_acc=-1.5_MTAU=entropy_alpha_M_ALPHA=1_batch128'
name = 'KLSAC'
name = 'KLSAC-T_load_acc=-1.5_MTAU=alphaEntropy=0.03_M_ALPHA=0.9_batch256'
name = 'KLSAC-T_load_acc=-1.5_MTAU=alphaEntropy(constant)=0.03_M_ALPHA=0.9_batch256'
name = 'KLSAC-T_load_acc=-1.5_alphaEntropy(constant)=M_ENTROPY=0.7_M_KL=0.5_batch256'

# name = f'KLRewardSAC-T_load_notfixedL5env2_KLweight=.5_trainset_1e5_acc=-1.5'
# name = f'KLSAC-T_load_acc=-1.5_MTAU=entropy_alpha_M_ALPHA=0.8_bs128'
# file = open(f'{SAVE_PATH}{name}/cp{cp}({r}).obj', 'rb')
# sim_outs2 = pickle.load(file)
# file = open(f'{SAVE_PATH}{name}/cp{cp}({r})_actions.obj', 'rb')
# actions, indices = pickle.load(file)

print(name)
# actions[:,1]
accs = np.asarray(actions)[:,1]
# accs*=0.6
steers = np.asarray(actions)[:,0]

In [ ]:
acc25 = sum([any(accs[indices[id]: indices[id+1]] > 3 * 0.1) for id in range(0, 99)])
acc15 = sum([1 if any(accs[indices[id]: indices[id] + 150] > 3 * 0.1) else 0 for id in range(0, 99)])
acc10 = sum([1 if any(accs[indices[id]: indices[id] + 100] > 3 * 0.1) else 0 for id in range(0, 99)])
acc5 = sum([1 if any(accs[indices[id]: indices[id] + 50] > 3 * 0.1) else 0 for id in range(0, 99)])
acc1 = sum([1 if any(accs[indices[id]: indices[id] + 10] > 3 * 0.1) else 0 for id in range(0, 99)])
acc01 = sum([1 if any(accs[indices[id]: indices[id] + 1] > 3 * 0.1) else 0 for id in range(0, 99)])
acc25, acc15, acc10, acc5, acc1, acc01

In [ ]:
acc25 = sum([any(accs[indices[id]: indices[id+1]] > 3 * 0.1) for id in range(0, 99)])
acc15 = sum([1 if any(accs[indices[id]: indices[id] + 150] > 3 * 0.1) else 0 for id in range(0, 99)])
acc10 = sum([1 if any(accs[indices[id]: indices[id] + 100] > 3 * 0.1) else 0 for id in range(0, 99)])
acc5 = sum([1 if any(accs[indices[id]: indices[id] + 50] > 3 * 0.1) else 0 for id in range(0, 99)])
acc1 = sum([1 if any(accs[indices[id]: indices[id] + 10] > 3 * 0.1) else 0 for id in range(0, 99)])
acc01 = sum([1 if any(accs[indices[id]: indices[id] + 1] > 3 * 0.1) else 0 for id in range(0, 99)])
acc25, acc15, acc10, acc5, acc1, acc01

In [ ]:
len(accs)

In [ ]:
acc25 = sum([any(accs[indices[id]: indices[id+1]] > 3 * 0.1) for id in range(0, 99)])
acc15 = sum([1 if any(accs[indices[id]: indices[id] + 150] > 3 * 0.1) else 0 for id in range(0, 99)])
acc10 = sum([1 if any(accs[indices[id]: indices[id] + 100] > 3 * 0.1) else 0 for id in range(0, 99)])
acc5 = sum([1 if any(accs[indices[id]: indices[id] + 50] > 3 * 0.1) else 0 for id in range(0, 99)])
acc1 = sum([1 if any(accs[indices[id]: indices[id] + 10] > 3 * 0.1) else 0 for id in range(0, 99)])
acc01 = sum([1 if any(accs[indices[id]: indices[id] + 1] > 3 * 0.1) else 0 for id in range(0, 99)])
acc25, acc15, acc10, acc5, acc1, acc01

In [ ]:
accs[indices[id]: indices[id] + 50] > 0.2

In [ ]:
from src.validate.validator import quantitative
quantitative(sim_outs2, actions)

In [ ]:
import  matplotlib.pyplot as plt

# plt.hist(accs, bins = 100)
import matplotlib.colors as mcolors
gamma = .3
plt.hist2d(accs, steers, bins = [50,50], norm=mcolors.PowerNorm(gamma))
# plt.xlim([-.6,.6])
# plt.ylim([-.08,.08])
# plt.hist(steers, 100)

In [ ]:
import  matplotlib.pyplot as plt

# plt.hist(accs, bins = 100)
import matplotlib.colors as mcolors
gamma = .3
plt.hist2d(accs, steers, bins = [50,50], norm=mcolors.PowerNorm(gamma))
# plt.xlim([-.6,.6])
# plt.ylim([-.08,.08])
# plt.hist(steers, 100)

In [ ]:
import matplotlib.pyplot as plt
plt.hist(accs, 100)

In [ ]:

cp = '130'
r = -51
name = f'SAC-T_load_trainset_2e5-1hist'
name = 'KLSAC-T_nonRLFT_fixedKLKin_fixedConfig_KLweight=1'
# cp = '160'
# r = -90
# name = 'KLSAC-T_load_acc=-1.5_alphaEntropy(constant)=M_ENTROPY=0.7_M_KL=0.5_batch256'

# name = f'KLRewardSAC-T_load_notfixedL5env2_KLweight=.5_trainset_1e5_acc=-1.5'
# name = f'KLSAC-T_load_acc=-1.5_MTAU=entropy_alpha_M_ALPHA=0.8_bs128'
# file = open(f'{SAVE_PATH}{name}/cp{cp}({r}).obj', 'rb')
# sim_outs2 = pickle.load(file)
file = open(f'{SAVE_PATH}{name}/cp{cp}({r})_actions.obj', 'rb')
actions_SAC, indices_SAC = pickle.load(file)
# name = 'BC-T-kin'
# file = open(f'{SAVE_PATH}{name}.obj', 'rb')
# sim_outs2 = pickle.load(file)
# file = open(f'{SAVE_PATH}{name}_actions.obj', 'rb')
# actions_SAC, indices_SAC = pickle.load(file)
print(name)
# actions[:,1]
accs_SAC_KL = np.asarray(actions_SAC)[:,1]
# accs*=0.6
steers_SAC_KL = np.asarray(actions_SAC)[:,0]

In [ ]:

import matplotlib.pyplot as plt
def draw(accs, accs_SAC, freq = False, label = False):
    plt.hist(accs, 100, label='Demo', weights=np.ones(len(accs))/len(accs))
    plt.hist(accs_SAC, 100, alpha=0.5, label='Policy',weights=np.ones(len(accs))/len(accs))
    if label:
        plt.legend(loc='upper right')
    if freq:
        plt.ylabel('Frequency (%)')
    plt.show()
    
# draw(accs, accs_SAC)
draw(accs, accs_SAC_KL)
# draw(steers, steers_SAC)
draw(steers, steers_SAC_KL)

In [ ]:
len(accs)


In [ ]:
import matplotlib.pyplot as plt
id = 13 # 130,131
id = 65 # 130,131
id = 2 # 4.5

midId = indices[id] + (indices[id+1] - indices[id])//2
plt.plot(range(len(accs[indices[id]:midId])), accs[indices[id]:midId])
plt.ylim(-0.6,0.6)
plt.axhline(y = 0.2, color = 'r', linestyle = '--')
plt.axhline(y = -0.2, color = 'r', linestyle = '--')
plt.show()
plt.ylim(-0.6,0.6)
plt.plot(range(len(accs[midId:indices[id+1]])), accs[midId:indices[id+1]])
plt.axhline(y = 0.2, color = 'r', linestyle = '--')
plt.axhline(y = -0.2, color = 'r', linestyle = '--')
plt.show()

In [ ]:
id = 2
plt.plot(range(len(steers[indices[id]:indices[id+1]])), steers[indices[id]:indices[id+1]])

In [ ]:
from l5kit.data import LocalDataManager
from l5kit.data.map_api import MapAPI
from l5kit.visualization.visualizer.zarr_utils import episode_out_to_visualizer_scene_gym_cle, simulation_out_to_visualizer_scene
from l5kit.configs import load_config_data
from bokeh.io import show
from l5kit.visualization.visualizer.visualizer import visualize
import torch
dmg = LocalDataManager(None)
env_config_path = '/home/pronton/rlhf-car/src/configs/gym_config.yaml'
cfg = load_config_data(env_config_path)
mapAPI = MapAPI.from_cfg(dmg, cfg)

def visualize_outputs(sim_outs):
    for sim_out in sim_outs: # for each scene
        vis_in = episode_out_to_visualizer_scene_gym_cle(sim_out, mapAPI)
        show(visualize(sim_out.scene_id, vis_in))

output_notebook()
visualize_outputs(sim_outs[0:3])

In [ ]:
from l5kit.data import LocalDataManager
from l5kit.data.map_api import MapAPI
from l5kit.visualization.visualizer.zarr_utils import episode_out_to_visualizer_scene_gym_cle, simulation_out_to_visualizer_scene
from l5kit.configs import load_config_data
from bokeh.io import show
from l5kit.visualization.visualizer.visualizer import visualize
import torch
dmg = LocalDataManager(None)
env_config_path = '/home/pronton/rlhf-car/src/configs/gym_config.yaml'
cfg = load_config_data(env_config_path)
mapAPI = MapAPI.from_cfg(dmg, cfg)

def visualize_outputs(sim_outs):
    for sim_out in sim_outs: # for each scene
        vis_in = episode_out_to_visualizer_scene_gym_cle(sim_out, mapAPI)
        show(visualize(sim_out.scene_id, vis_in))

output_notebook()
visualize_outputs(sim_outs2[6:8])